In [1]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from fastai.vision.gan import *
import string

from PIL import Image, ImageDraw, ImageFont
import PIL

In [2]:
path = Path(r'C:\Users\akash\.fastai\data\imagenette-160')

In [3]:
def get_crit_data(classes, bs, size):
    src = ImageList.from_folder(path, include=classes).split_by_rand_pct(0.1, seed=42)
    ll = src.label_from_folder(classes=classes)
    data = (ll.transform(get_transforms(max_zoom=2.), size=size)
           .databunch(bs=bs).normalize(imagenet_stats))
    data.c = 3
    return data

In [4]:
data_crit = get_crit_data(['image_gen', 'images'], bs=16, size=160)

In [5]:
data_crit

ImageDataBunch;

Train: LabelList (22871 items)
x: ImageList
Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160)
y: CategoryList
images,images,images,images,images
Path: C:\Users\akash\.fastai\data\imagenette-160;

Valid: LabelList (2541 items)
x: ImageList
Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160),Image (3, 160, 160)
y: CategoryList
image_gen,images,image_gen,image_gen,images
Path: C:\Users\akash\.fastai\data\imagenette-160;

Test: None

In [7]:
loss_critic = AdaptiveLoss(nn.BCEWithLogitsLoss())

In [8]:
wd = 1e-3
def create_critic_learner(data, metrics):
    return Learner(data, gan_critic(), metrics=metrics, loss_func=loss_critic, wd=wd)


learn_critic = create_critic_learner(data_crit, accuracy_thresh_expand)

In [10]:

learn_critic.fit_one_cycle(6, 1e-3)

epoch,train_loss,valid_loss,accuracy_thresh_expand,time
0,0.692687,0.691680,0.528139,27:10
1,0.690450,0.693687,0.528139,27:24
2,0.691885,0.691575,0.528139,27:17
3,0.689878,0.691569,0.528139,27:12
4,0.692809,0.691593,0.528139,27:13
5,0.692399,0.691593,0.528139,27:12


In [11]:
learn_critic.export('critic.pkl')